# Variational Auto Encoder
---
Details from [here](https://arxiv.org/abs/1312.6114). Implementation derived from [here](https://github.com/pytorch/examples/tree/master/vae).

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

BATCH_SIZE = 128
EPOCHS = 10
LOG_INTERVAL = 100
CUDA = torch.cuda.is_available()

torch.manual_seed(1)

device = torch.device("cuda" if CUDA else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

# VAE Model

In [2]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [3]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

$KL = \frac{1}{2} \Sigma(1 + log(\sigma^{2}) - \mu^{2} - \sigma^{2})$

In [4]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [5]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [6]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(BATCH_SIZE, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'Results/VAE Results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [7]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'Results/VAE Results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 550.187805
Train Epoch: 1 [12800/60000 (21%)]	Loss: 182.705048
Train Epoch: 1 [25600/60000 (43%)]	Loss: 147.834747
Train Epoch: 1 [38400/60000 (64%)]	Loss: 142.742676
Train Epoch: 1 [51200/60000 (85%)]	Loss: 131.814316
====> Epoch: 1 Average loss: 164.1690
====> Test set loss: 120.0071
Train Epoch: 2 [0/60000 (0%)]	Loss: 128.557312
Train Epoch: 2 [12800/60000 (21%)]	Loss: 127.930389
Train Epoch: 2 [25600/60000 (43%)]	Loss: 119.412476
Train Epoch: 2 [38400/60000 (64%)]	Loss: 120.694443
Train Epoch: 2 [51200/60000 (85%)]	Loss: 116.721458
====> Epoch: 2 Average loss: 121.5907
====> Test set loss: 107.8307
Train Epoch: 3 [0/60000 (0%)]	Loss: 118.998482
Train Epoch: 3 [12800/60000 (21%)]	Loss: 117.597969
Train Epoch: 3 [25600/60000 (43%)]	Loss: 115.170647
Train Epoch: 3 [38400/60000 (64%)]	Loss: 115.573395
Train Epoch: 3 [51200/60000 (85%)]	Loss: 114.826523
====> Epoch: 3 Average loss: 114.4107
====> Test set loss: 103.2697
Train Epoch: 4 [0/60000 (0%)]	L

KeyboardInterrupt: 